In [13]:
# Useful links: 
# https://developers.google.com/optimization/mip/mip_example

In [14]:
#!pip install ortools

In [15]:
# utility file used to generate mzn data files from the dat files
import numpy as np
# open the file in Instances folder
f = open("Instances/inst04.dat", "r")
# the first line is the number of couriers
m = int(f.readline())
# the second line is the number of items
n = int(f.readline())
# the third line is the load size of each courier
load_size = [int(x) for x in f.readline().split()]
# the fourth line is the size of each item
item_size = [int(x) for x in f.readline().split()]
# the rest is the distance matrix
distance = []
for i in range(n+1):
    distance.append([int(x) for x in f.readline().split()])
# close the file
f.close()
print("couriers:", m)
print("items:", n)
print("load_size:", load_size)
print("item_size:", item_size)
# output the distance matrix as a numpy array
distance = np.array(distance)
print("distance:\n", distance)

couriers: 8
items: 10
load_size: [200, 180, 160, 185, 180, 200, 160, 200]
item_size: [10, 25, 18, 16, 14, 16, 24, 19, 23, 19]
distance:
 [[  0  56  86  77  81 128 107 154  70  93  53]
 [ 56   0  79  31  62  87  61 107  37  37  24]
 [ 86  79   0 109  17  43 110  68  43  69  55]
 [ 87  31 109   0  92 116  30  77  66  40  55]
 [ 81  62  17  92   0  47  93  82  26  52  38]
 [128  87  43 116  47   0 117  52  58  76  75]
 [116  61 110  30  93 117   0  65  67  41  63]
 [163 107  78  77  82  52  65   0  93  70 110]
 [ 70  41  43  66  26  73  67  93   0  26  17]
 [ 93  37  69  40  52  76  41  70  26   0  40]
 [ 53  24  55  54  38  75  63 110  17  40   0]]


In [16]:
from ortools.linear_solver import pywraplp

def solve_courier_problem(courier_capacities, item_sizes, item_distances):
    num_couriers = len(courier_capacities)
    num_items = len(item_sizes)

    solver = pywraplp.Solver.CreateSolver('SCIP')
    max_distance = solver.NumVar(0, solver.infinity(), 'max_distance')

    # Create variables
    assignment = [[[solver.IntVar(0, 1, f'assignment_{i}_{j}_{k}') for j in range(num_items + 1)] for k in range(num_items + 1)] for i in range(num_couriers)]
    
    # CHECKED The diagonal of the matrix is 0
    solver.Add(sum(assignment[i][j][j] for i in range(num_couriers) for j in range(num_items + 1)) == 0)
    
    # Create constraints: items are assigned to at most one courier
    for j in range(num_items):
        solver.Add(sum(assignment[i][j][k] for i in range(num_couriers) for k in range(num_items + 1)) == 1)
        solver.Add(sum(assignment[i][k][j] for i in range(num_couriers) for k in range(num_items + 1)) == 1)
    
    for i in range(num_couriers):
        # We set boundaries for max distance, so that we can minimize it later
        solver.Add(sum(assignment[i][j][k] * item_distances[j][k] for j in range(num_items + 1) for k in range(num_items + 1)) <= max_distance)
        # Create constraints: each courier carries at least one item
        solver.Add(sum(assignment[i][num_items][k] for k in range(num_items + 1)) == 1)
        # Create constraint: courier capacities are respected
        solver.Add(sum(assignment[i][j][k] * item_sizes[j] for j in range(num_items) for k in range(num_items + 1)) <= courier_capacities[i])
        for j in range(num_items + 1):
            # Create constraint: n arcs in, n arcs out
            solver.Add(sum(assignment[i][j][k] for k in range(num_items + 1)) == sum(assignment[i][k][j] for k in range(num_items + 1)))
    
    # Create constraint: Eliminate subtours using the Miller-Tucker-Zemlin (MTZ) formulation
    # for i in range(num_couriers):
    #     # Count the number of items assigned to this courier
    #     num_assigned_items = sum(assignment[i][j][k].solution_value() for j in range(num_items) for k in range(num_items + 1))
    #     # If the number of assigned items is greater than 1, apply subtour elimination constraint
    #     if num_assigned_items > 1:
    
# for i in range(m):
#     for j in range(n+1):
#         for k in range(n+1):
#             if j != n and j != k:
#                 model.constraints.add(model.u[i, j] - model.u[i, k] + n * model.roots[i, j, k] <= n - 1)
    # Create constraint: Eliminate subtours using the Miller-Tucker-Zemlin (MTZ) formulation
    #model.u = Var(range(m), range(n+1), within=NonNegativeIntegers)
    # I create a 2d array of variables u[i][j] where i is the courier and j is the item
    u = [[solver.IntVar(0, solver.infinity(), f'u_{i}_{j}') for j in range(num_items + 1)] for i in range(num_couriers)]

    for i in range(num_couriers):
        for j in range(0, num_items + 1):
            for k in range(0, num_items + 1):
                if j != k and j != num_items:
                    # I add the constraint of the MTZ formulation
                    solver.Add(u[i][j] - u[i][k] + 1 <= (num_items - 1) * (1 - assignment[i][j][k]))
                    #solver.Add(u[i][j] - u[i][k] + num_items * assignment[i][j][k] <= num_items - 1)
    
    solver.Minimize(max_distance)
    
    solver.set_time_limit(300000)
    
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        for i in range(num_couriers):
            print(f'Courier {i} takes item: ')
            print('{')
            for j in range(num_items):
                for k in range(num_items + 1):
                    if assignment[i][j][k].solution_value() > 0:
                        print(f' -> {j}')
            print('}')
        print('Max distance:', max_distance.solution_value())
        print('Objective value:', solver.Objective().Value())
        # I print the values of assignment
        # for each courier i create and print a item matrix
        print()
        for i in range(num_couriers):
            print(f'Courier {i} item matrix:')
            for j in range(num_items + 1):
                for k in range(num_items + 1):
                    print(int(assignment[i][j][k].solution_value()), end=' ')
                print()
            print()
        # I print the values of u
        print()
        print('u matrix:')
        for i in range(num_couriers):
            for j in range(num_items + 1):
                print(int(u[i][j].solution_value()), end=' ')
            print()
        print(item_distances)
    else:
        print('The problem does not have an optimal solution.')

solve_courier_problem(load_size, item_size, distance)


Solution:
Courier 0 takes item: 
{
 -> 9
}
Courier 1 takes item: 
{
 -> 7
}
Courier 2 takes item: 
{
 -> 5
}
Courier 3 takes item: 
{
 -> 8
}
Courier 4 takes item: 
{
 -> 6
}
Courier 5 takes item: 
{
 -> 0
}
Courier 6 takes item: 
{
 -> 1
 -> 3
}
Courier 7 takes item: 
{
 -> 2
 -> 4
}
Max distance: 220.0
Objective value: 220.0

Courier 0 item matrix:
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 1 
0 0 0 0 0 0 0 0 0 1 0 

Courier 1 item matrix:
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 1 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 0 0 0 

Courier 2 item matrix:
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0